In [58]:
import librosa
import matplotlib as plt
import numpy as np
import pandas as pd

from pathlib import Path
import os

#!pip install pydub

from pydub import AudioSegment
from pydub.utils import make_chunks

In [59]:
features = [
    'filename',
    'length',
    'chroma_stft_mean',
    'chroma_stft_var',
    'rms_mean',
    'rms_var',
    'spectral_centroid_mean',
    'spectral_centroid_var',
    'spectral_bandwidth_mean',
    'spectral_bandwidth_var',
    'rolloff_mean',
    'rolloff_var',
    'zero_crossing_rate_mean',
    'zero_crossing_rate_var',
    'harmony_mean',
    'harmony_var',
    'tempo',
    'mfcc1_mean',
    'mfcc1_var',
    'mfcc2_mean',
    'mfcc2_var',
    'mfcc3_mean',
    'mfcc3_var',
    'mfcc4_mean',
    'mfcc4_var',
    'mfcc5_mean',
    'mfcc5_var',
    'mfcc6_mean',
    'mfcc6_var',
    'mfcc7_mean',
    'mfcc7_var',
    'mfcc8_mean',
    'mfcc8_var',
    'mfcc9_mean',
    'mfcc9_var',
    'mfcc10_mean',
    'mfcc10_var',
    'mfcc11_mean',
    'mfcc11_var',
    'mfcc12_mean',
    'mfcc12_var',
    'mfcc13_mean',
    'mfcc13_var',
    'mfcc14_mean',
    'mfcc14_var',
    'mfcc15_mean',
    'mfcc15_var',
    'mfcc16_mean',
    'mfcc16_var',
    'mfcc17_mean',
    'mfcc17_var',
    'mfcc18_mean',
    'mfcc18_var',
    'mfcc19_mean',
    'mfcc19_var',
    'mfcc20_mean',
    'mfcc20_var'
]

In [60]:
labels = [
    'blues',
    'classical',
    'country',
    'disco',
    'hiphop',
    'jazz',
    'kpop',
    'metal',
    'pop',
    'reggae',
    'rock'
]

In [140]:
for label in labels:
    for i in range(0,100):
        
        file = "Data/genres_original/{}/{}.000{}.wav".format(label,label, i if i>9 else "0"+str(i))
        #print(file)
                
        if (file != "Data/genres_original/jazz/jazz.00054.wav"): # jazz 54 is corrupt?
            
            myaudio = AudioSegment.from_file(file , "wav") 
            chunk_length_ms = 3000 # millisec
            chunks = make_chunks(myaudio, chunk_length_ms)

            for i, chunk in enumerate(chunks):
                chunk_name = "{}_{}.wav".format(file.replace(".wav","").replace("genres_original","genres_3sec"),i)
                if("_10" not in chunk_name):
                    #print ("exporting", chunk_name)
                    chunk.export(chunk_name, format="wav")

collect features:

In [62]:
def get_features(file):
    y, sr = librosa.load(file)

    # values:
    length = librosa.get_duration(y=y, sr=sr)
    
    chroma_stft_mean = librosa.feature.chroma_stft(y=y, sr=sr).mean()
    chroma_stft_var = librosa.feature.chroma_stft(y=y, sr=sr).var()
    
    rms_mean = librosa.feature.rms(y=y).mean()
    rms_var = librosa.feature.rms(y=y).var()
    
    spectral_centroid_mean = librosa.feature.spectral_centroid(y=y, sr=sr).mean()
    spectral_centroid_var = librosa.feature.spectral_centroid(y=y, sr=sr).var()
    
    spectral_bandwidth_mean = librosa.feature.spectral_bandwidth(y=y, sr=sr).mean()
    spectral_bandwidth_var = librosa.feature.spectral_bandwidth(y=y, sr=sr).var()
    
    rolloff_mean = librosa.feature.spectral_rolloff(y=y, sr=sr).mean()
    rolloff_var = librosa.feature.spectral_rolloff(y=y, sr=sr).var()
    
    zero_crossing_rate_mean = librosa.feature.zero_crossing_rate(y).mean()
    zero_crossing_rate_var = librosa.feature.zero_crossing_rate(y).var()

    harmonic,percussive = librosa.effects.hpss(y)
    harmony_mean = np.mean(harmonic)
    harmony_var = np.var(harmonic)

    onset_env = librosa.onset.onset_strength(y=y, sr=sr)
    tempo = librosa.feature.tempo(onset_envelope=onset_env, sr=sr)[0]

    record_values = [
        file ,
        length ,
        chroma_stft_mean ,
        chroma_stft_var ,
        rms_mean ,
        rms_var ,
        spectral_centroid_mean ,
        spectral_centroid_var ,
        spectral_bandwidth_mean ,
        spectral_bandwidth_var ,
        rolloff_mean ,
        rolloff_var ,
        zero_crossing_rate_mean ,
        zero_crossing_rate_var ,
        
        harmony_mean,
        harmony_var,
        
        tempo
    ]

    for i in range(1,21):
        mfcci_mean = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=i).mean()
        mfcci_var = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=i).var()
        record_values.append(mfcci_mean)
        record_values.append(mfcci_var)

    #print(record_values, len(record_values))

    record = pd.DataFrame(
        [ record_values ],
        columns=features
    )
    
    return record

In [144]:
collected_samples = pd.DataFrame(data=None, columns=(features + ['label']) )

for label in labels:
    print(label)
    for filename in os.listdir("Data/genres_3sec/{}/".format(label)):
        f = os.path.join("Data/genres_3sec/{}/".format(label), filename)
        rec = get_features(f)
        rec['label'] = f.split("/")[2]
        rec['filename'] = f.split("/")[3]
        collected_samples = pd.concat([collected_samples, rec])
        #print(collected_samples)
        
collected_samples


blues


C:\Users\julcb\AppData\Local\Temp\ipykernel_30628\877779240.py:10: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  collected_samples = pd.concat([collected_samples, rec])


classical
country
disco
hiphop
jazz
metal
pop
reggae
rock


,filename,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,blues.00000_0.wav,3.0,0.335555,0.090997,0.130189,0.003559,1773.358004,169450.829520,1972.334258,117272.640189,...,2362.774902,1.548628,2227.368652,1.501281,2105.762207,1.157216,1998.835205,1.087376,1901.177490,blues
0,blues.00000_1.wav,3.0,0.343523,0.086782,0.112119,0.001491,1817.244034,90766.297254,2010.751494,65940.666243,...,2452.354980,1.868640,2314.378906,1.771890,2188.797607,1.530792,2079.760742,1.742847,1979.645386,blues
0,blues.00000_2.wav,3.0,0.347746,0.092495,0.130895,0.004552,1790.722357,110071.206973,2088.184750,73391.498001,...,2505.148438,2.791831,2360.711182,2.766150,2232.194336,2.519214,2118.593506,2.516604,2014.322266,blues
0,blues.00000_3.wav,3.0,0.363863,0.087207,0.131349,0.002338,1660.545231,109496.936296,1967.920582,79805.901351,...,2439.380615,1.901317,2299.581299,1.859761,2173.838623,1.577565,2063.523438,1.677718,1962.153809,blues
0,blues.00000_4.wav,3.0,0.335481,0.088482,0.142370,0.001734,1634.465077,77425.419232,1954.633566,57359.695604,...,2281.655762,3.439414,2150.558105,3.336730,2034.118408,2.984012,1930.689331,2.859887,1835.903076,blues
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,rock.00099_5.wav,3.0,0.348781,0.080553,0.049680,0.000098,1503.238681,163419.049525,1725.514991,85476.555705,...,4923.151855,-2.634636,4638.445312,-2.722517,4382.626953,-2.878276,4154.901855,-2.648109,3950.097412,rock
0,rock.00099_6.wav,3.0,0.372556,0.082633,0.057615,0.000096,1852.706840,277236.230048,1914.481257,97378.034054,...,3708.217773,-3.813271,3498.619873,-3.775346,3307.222656,-4.203818,3139.811279,-3.974402,2984.764160,rock
0,rock.00099_7.wav,3.0,0.347815,0.089140,0.051906,0.000674,1348.383673,662359.245401,1566.961070,137898.245964,...,7897.867676,-5.969096,7435.306152,-5.362641,7029.924316,-4.989933,6664.940918,-4.751178,6334.845703,rock
0,rock.00099_8.wav,3.0,0.386801,0.084649,0.065967,0.000312,2086.306423,200732.442407,2021.144027,20772.677177,...,3130.227539,-1.595695,2947.988037,-1.150531,2788.798340,-0.966437,2643.941895,-0.884805,2512.499756,rock


In [145]:
for label in ['kpop']:
    print(label)
    for filename in os.listdir("Data/genres_3sec/{}/".format(label)):
        f = os.path.join("Data/genres_3sec/{}/".format(label), filename)
        rec = get_features(f)
        rec['label'] = f.split("/")[2]
        rec['filename'] = f.split("/")[3]
        collected_samples = pd.concat([collected_samples, rec])
        #print(collected_samples)
        
collected_samples

kpop


,filename,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,blues.00000_0.wav,3.0,0.335555,0.090997,0.130189,0.003559,1773.358004,1.694508e+05,1972.334258,117272.640189,...,2362.774902,1.548628,2227.368652,1.501281,2105.762207,1.157216,1998.835205,1.087376,1901.177490,blues
0,blues.00000_1.wav,3.0,0.343523,0.086782,0.112119,0.001491,1817.244034,9.076630e+04,2010.751494,65940.666243,...,2452.354980,1.868640,2314.378906,1.771890,2188.797607,1.530792,2079.760742,1.742847,1979.645386,blues
0,blues.00000_2.wav,3.0,0.347746,0.092495,0.130895,0.004552,1790.722357,1.100712e+05,2088.184750,73391.498001,...,2505.148438,2.791831,2360.711182,2.766150,2232.194336,2.519214,2118.593506,2.516604,2014.322266,blues
0,blues.00000_3.wav,3.0,0.363863,0.087207,0.131349,0.002338,1660.545231,1.094969e+05,1967.920582,79805.901351,...,2439.380615,1.901317,2299.581299,1.859761,2173.838623,1.577565,2063.523438,1.677718,1962.153809,blues
0,blues.00000_4.wav,3.0,0.335481,0.088482,0.142370,0.001734,1634.465077,7.742542e+04,1954.633566,57359.695604,...,2281.655762,3.439414,2150.558105,3.336730,2034.118408,2.984012,1930.689331,2.859887,1835.903076,blues
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,kpop.-995.wav,3.0,0.479903,0.099387,0.282667,0.008128,2149.837595,6.106146e+05,2581.762352,251934.235486,...,1322.912231,8.862573,1258.029419,8.474761,1194.835449,7.763658,1145.807861,7.582238,1094.655273,kpop
0,kpop.-996.wav,3.0,0.528338,0.085169,0.265109,0.012062,1950.183596,6.100136e+05,2449.874414,286483.820574,...,1603.543579,8.371280,1522.635498,8.232771,1441.412476,7.548554,1379.748169,7.508713,1314.504761,kpop
0,kpop.-997.wav,3.0,0.541197,0.092248,0.272684,0.009292,2067.502024,5.572963e+05,2545.651376,237826.048144,...,1371.700928,8.781055,1302.517822,8.523456,1236.630981,7.919319,1183.685913,7.887958,1130.318604,kpop
0,kpop.-998.wav,3.0,0.537191,0.098024,0.250019,0.013746,2231.741035,1.459891e+06,2616.655679,402349.320622,...,1918.280273,4.074778,1810.315063,3.823031,1715.845703,3.199282,1636.146362,3.383192,1559.539673,kpop


In [146]:
#collected_new.to_csv("Data/collected_features.csv", index=False)

collected_samples.to_csv("Data/collected_samples.csv", index=False)

In [147]:
collected_samples_path = "Data/collected_samples.csv"
collected_samples_pd = pd.read_csv(collected_samples_path)
collected_samples_pd.head()

,filename,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,blues.00000_0.wav,3.0,0.335555,0.090997,0.130189,0.003559,1773.358004,169450.829520,1972.334258,117272.640189,...,2362.7750,1.548628,2227.3687,1.501281,2105.7622,1.157216,1998.8352,1.087376,1901.1775,blues
1,blues.00000_1.wav,3.0,0.343523,0.086782,0.112119,0.001491,1817.244034,90766.297254,2010.751494,65940.666243,...,2452.3550,1.868640,2314.3790,1.771890,2188.7976,1.530792,2079.7607,1.742847,1979.6454,blues
2,blues.00000_2.wav,3.0,0.347746,0.092495,0.130895,0.004552,1790.722357,110071.206973,2088.184750,73391.498001,...,2505.1484,2.791831,2360.7112,2.766150,2232.1943,2.519214,2118.5935,2.516604,2014.3223,blues
3,blues.00000_3.wav,3.0,0.363863,0.087207,0.131349,0.002338,1660.545231,109496.936296,1967.920582,79805.901351,...,2439.3806,1.901317,2299.5813,1.859761,2173.8386,1.577565,2063.5234,1.677718,1962.1538,blues
4,blues.00000_4.wav,3.0,0.335481,0.088482,0.142370,0.001734,1634.465077,77425.419232,1954.633566,57359.695604,...,2281.6558,3.439414,2150.5580,3.336730,2034.1184,2.984012,1930.6893,2.859887,1835.9031,blues
